In [ ]:
# for i in range(10):
#     for j in range(10):
#         for k in range(10):
#             if (i%5 == 0) & (j%5  == 0) & (k%5 == 0): 
#                 print(i, j, k, "matched")
#                 break
#             else: print(f"i = {i}, j = {j}, k = {k}")


0 0 0 matched
i = 0, j = 1, k = 0
i = 0, j = 1, k = 1
i = 0, j = 1, k = 2
i = 0, j = 1, k = 3
i = 0, j = 1, k = 4
i = 0, j = 1, k = 5
i = 0, j = 1, k = 6
i = 0, j = 1, k = 7
i = 0, j = 1, k = 8
i = 0, j = 1, k = 9
i = 0, j = 2, k = 0
i = 0, j = 2, k = 1
i = 0, j = 2, k = 2
i = 0, j = 2, k = 3
i = 0, j = 2, k = 4
i = 0, j = 2, k = 5
i = 0, j = 2, k = 6
i = 0, j = 2, k = 7
i = 0, j = 2, k = 8
i = 0, j = 2, k = 9
i = 0, j = 3, k = 0
i = 0, j = 3, k = 1
i = 0, j = 3, k = 2
i = 0, j = 3, k = 3
i = 0, j = 3, k = 4
i = 0, j = 3, k = 5
i = 0, j = 3, k = 6
i = 0, j = 3, k = 7
i = 0, j = 3, k = 8
i = 0, j = 3, k = 9
i = 0, j = 4, k = 0
i = 0, j = 4, k = 1
i = 0, j = 4, k = 2
i = 0, j = 4, k = 3
i = 0, j = 4, k = 4
i = 0, j = 4, k = 5
i = 0, j = 4, k = 6
i = 0, j = 4, k = 7
i = 0, j = 4, k = 8
i = 0, j = 4, k = 9
0 5 0 matched
i = 0, j = 6, k = 0
i = 0, j = 6, k = 1
i = 0, j = 6, k = 2
i = 0, j = 6, k = 3
i = 0, j = 6, k = 4
i = 0, j = 6, k = 5
i = 0, j = 6, k = 6
i = 0, j = 6, k = 7
i = 0, j = 6

In [3]:
import os
import pandas as pd
import numpy as np


In [4]:

seed_value = 42
np.random.seed(seed_value)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Movie Selection

In [5]:
# display the aspect of a movie
# display the nodes and ask for node-edge relation authenticity 
# display question from the aspects and ask for the meaningfulness and complexity level of the question.
# save the authenticity score, question meaningfulness and complexity score with the ground truth value


# process for each category:
# -------------------------------
# 1. sort the movie files from the dataset folder aspectwise
# 2. preprocess the files based on the criteria we followed 
# 3. chose 30 percent of the non empty files with atleast 10 node-edge pairs and 10 questions using random sampling.
# 4. choose on file with aspect, fetch its aspect information from the wikipedia page
# 5. display its aspect content and then one by one present the node-edge pairs for authenticity test
# 6. for each pair, receive the response over five scale and save the responses in a dictionary key will be the scale value and the value will be the count of the scale.
# 7. after the nodes evaluation of the movie, display the 30 percent question sampled randomly.
# 8. display the question, get the response of complexity test and save the match-mismatch score hopwise,questionwise. save the response in a dictionary with the proper keys and values.
# 9. save the dictionary and file names already checked.


In [6]:
from bs4 import BeautifulSoup
import requests
from pathlib import Path
from tqdm import tqdm

In [5]:
    
def save_aspect(aspect, content, movie_name, yor, save_file_path = "/mnt/Data/prabirmondal/prabir/python_program/movie_sense/SRI_KG/Movie_sense_KG/Movie_sense_KG/Analysis/movie.txt"):
    # print(f"from save aspect, {aspect}")
    introduction = f"Movie_name = {movie_name}, Year or release = {yor}.\n HERE IS THE DETAILS OF MOVIE'S {aspect.upper()}: \n"
    underline = "-----------------------------------------------------\n"
    para = introduction + underline + content
    # print(para)
    # input()
    # print(para)
    # Open the file in write mode and save the paragraph
    with open(save_file_path, 'w') as file:
        file.write(para)
    
    # print(f"Paragraph saved to {save_file_path} for {aspect}.")


def scrape(wiki_link):
    Aspects = [
        'Plot',
        'Cast',
        'Production',
        'Music',
        'Soundtrack',
        'Themes',
        'Accolades',
        ]

    Aspects = [aspect.lower() for aspect in Aspects]
    
    # Send a request to the Wikipedia page
    response = requests.get(wiki_link)
    
    # Parse the page content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    scraped_dict = {}
    
    #-------------------Summary scraping--------------------------
    # Find the first paragraph after the title, which is usually the summary
    summary_paragraphs = []
    # Wikipedia's summary paragraphs are inside <p> tags but before any <h2> tag
    for paragraph in soup.find_all('p'):
        
        # Ensure the paragraph has text and is not empty
        if paragraph.get_text().strip():
            summary_paragraphs.append(paragraph.get_text().strip())
        
        # Stop once we hit the first section heading (e.g. 'Plot' or 'Contents')
        if paragraph.find_next_sibling(['h2', 'h3']):
            break
    
    scraped_dict["summary"] =  ' '.join(summary_paragraphs)
    
    
    # -------------------Other Aspect Scraping---------------------
    Headings = soup.find_all('div', class_ = 'mw-heading mw-heading2')
    for heading in Headings:
        try:
            aspect, _ = (heading.text).split('[')
        except:
            aspect = heading.text
            
        if aspect.lower() in Aspects:
            next_siblings = heading.find_next_siblings()
            text = ''
            for next_sibling in next_siblings:
                next_sibling_name = next_sibling.name
                # sub_sibling = ''
                # print(f"............next_sibling_name = {next_sibling_name}")
                if (next_sibling_name =='style'):
                    continue
                
                elif (next_sibling_name == 'div'):
                    clss = next_sibling.get('class')
                    
                    if ('mw-heading2' in clss):  # break because heading ended
                        break
                text += " "+ next_sibling.text
            scraped_dict[aspect] = text 
    return scraped_dict

In [1]:
def preprocess_node_df(df):
    # Define a function to check if the edge contains both node names and at least 5 words
    def is_valid_edge(row):
        edge_words = row['edge'].split()
        return (
            len(edge_words) >= 5 and 
            row['node_1'] in row['edge'] and 
            row['node_2'] in row['edge']
        )
    
    # Filter the dataframe using the validation function
    filtered_df = df[df.apply(is_valid_edge, axis=1)].reset_index(drop=True)
    
    return filtered_df

def randomPick_question(df):
    # Create an empty dataframe
    random_row = pd.DataFrame()
    
    # Create an empty dataframe with predefined columns
    random_row = pd.DataFrame(columns=['Column1', 'Column2', 'Column3'])
    
    total_question = len(df)
    if total_question >= 10:
        sample_30_count = int(0.3*total_question)
        # Randomly select a single row
        random_row = df.sample(n=sample_30_count, random_state=seed_value)
        # random_rows = df.sample(n=n, random_state=seed_value)
    
    return random_row

def read_csv(file_path:str):
    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
        except:
            df = pd.DataFrame()
    else:
        df = pd.DataFrame()
        
    return df
    

def fetch_questions(category, node_file_name, hop):
    questionFile_root = "/mnt/Data/prabirmondal/prabir/python_program/movie_sense/SRI_KG/Movie_sense_KG/Movie_sense_KG/Dataset/Questions_Answers/Hollywood"
    
    hop_yes_no_path = os.path.join(questionFile_root, category, hop, "yes_no", node_file_name)
    hop_MCQ_S_path = os.path.join(questionFile_root, category, hop, "MCQ_single_correct", node_file_name)
    hop_MCQ_M_path = os.path.join(questionFile_root, category, hop, "MCQ_Multiple_correct", node_file_name)
    
    
    hop_yes_no_questions = randomPick_question(read_csv(hop_yes_no_path))
    hop_MCQ_S_questions = randomPick_question(read_csv(hop_MCQ_S_path))
    hop_MCQ_M_questions = randomPick_question(read_csv(hop_MCQ_M_path))
    
    # print("from fetch_questions")
    # print("-----------------------------")
    # print(len(hop_yes_no_questions))
    # print(len(hop_MCQ_S_questions))
    # print(len(hop_MCQ_M_questions))
    
    return hop_yes_no_questions, hop_MCQ_S_questions, hop_MCQ_M_questions


# Function to clear the terminal
def clear_terminal():
    os.system('cls' if os.name == 'nt' else 'clear')
    

In [ ]:
columns = ["YoR", "movie_name", "imdb_rating", "wiki_link", "popular"]
movie_links = pd.read_excel("/mnt/Data/prabirmondal/prabir/python_program/movie_sense/SRI_KG/Movie_sense_KG/Movie_sense_KG/aspect_wise_knowledge_graph/workspace/Movie_list.xlsx", sheet_name = "hollywood", engine='openpyxl')
movie_links.columns = columns


In [ ]:
# node evaluation
#------------------------
def eval_nodes(node_file, triplet_eval):    
    def triplet_eval_response(row):
        response_list,triplet_rspns = [1, 0], None
        while triplet_rspns not in response_list:
            clear_terminal()
            print(f"Node_1 : {row['node_1']}, \nNode_2 : {row['node_2']}, \nRelation : {row['edge']}")
            print("===================================================================================")
            triplet_rspns = int(input("Is the triplet from the context? Press 1 for Yes, 0 for No: "))
            if triplet_rspns not in response_list:
                print("Wrong Entry, please enter again. Press any key to re-enter.")
                input()
        return triplet_rspns
    
    def triplet_relation_eval_response(row):
        response_list,triplet_rspns = [1, 0], None
        while triplet_rspns not in response_list:
            clear_terminal()
            print(f"Node_1 : {row['node_1']}, \nNode_2 : {row['node_2']}, \nRelation : {row['edge']}")
            print("===================================================================================")
            triplet_rspns = int(input("Is the relation true between the two nodes? Press 1 for Yes, 0 for No: "))
            if triplet_rspns not in response_list:
                print("Wrong Entry, please enter again. Press any key to re-enter.")
                input()
        return triplet_rspns
        
        

    
    for index, row in node_file.iterrows():        
        triplet_eval_response = triplet_eval_response(row)
        if triplet_eval_response == 1:
            triplet_eval["triplet_relevant"] += 1           
            triplet_relation_eval_response = triplet_relation_eval_response(row)
            if triplet_relation_eval_response == 1:
                triplet_eval["relation_relevant"] += 1
            else: triplet_eval["relation_irrelevant"] += 1
            
        else: triplet_eval["triplet_irrelevant"] += 1
        
    return triplet_eval

In [ ]:
# for question evaluation
#------------------------------
def eval_question(yes_no_questions, MCQ_S_questions, MCQ_M_questions, eval_dataframe_yes_no, eval_dataframe_MCQ_S, eval_dataframe_MCQ_M):
    def is_from_aspect(row, question_col, answer_col):
        response_list,qstn_rspns = [1, 0], None
        while qstn_rspns not in response_list:
            clear_terminal()
            print(f"Question : {row[question_col]}, \nAnswer : {row[answer_col]}")
            print("===================================================================================")
            qstn_rspns = int(input("Is the question from the aspect? Press 1 for Yes, 0 for No: "))
            if qstn_rspns not in response_list:
                print("Wrong Entry, please enter again. Press any key to re-enter.")
                input()        
        return qstn_rspns
    
    
    def is_meaningful(row, question_col, answer_col):
        response_list,qstn_rspns = [2, 1, 0], None
        while qstn_rspns not in response_list:
            clear_terminal()
            print(f"Question : {row[question_col]}, \nAnswer : {row[answer_col]}")
            print("===================================================================================")
            qstn_rspns = int(input("Is the question Meaningful? Press 2 for Strongly meaningful, 1 for Moderate meaningful, 0 for Meaningless: "))
            if qstn_rspns not in response_list:
                print("Wrong Entry, please enter again. Press any key to re-enter.")
                input()        
        return qstn_rspns
    
    def is_answer_correct(row, question_col, answer_col):
        response_list,qstn_rspns = [2, 1, 0], None
        while qstn_rspns not in response_list:
            clear_terminal()
            print(f"Question : {row[question_col]}, \nAnswer : {row[answer_col]}")
            print("===================================================================================")
            qstn_rspns = int(input("Is the answer correct? Press 2 for Very correct, 1 for partially correct, 0 for incorrect: "))
            if qstn_rspns not in response_list:
                print("Wrong Entry, please enter again. Press any key to re-enter.")
                input()        
        return qstn_rspns
    
    def question_complexity(row, question_col, answer_col):
        response_list,qstn_rspns = [2, 1, 0], None
        while qstn_rspns not in response_list:
            clear_terminal()
            print(f"Question : {row[question_col]}, \nAnswer : {row[answer_col]}")
            print("===================================================================================")
            qstn_rspns = int(input("How complex the question is? Press 2 for Very complex, 1 for Moderate, 0 for Simple: "))
            if qstn_rspns not in response_list:
                print("Wrong Entry, please enter again. Press any key to re-enter.")
                input()        
        return qstn_rspns
    
    
    def single_question_eval(row, question_col, answer_col, eval_dataframe):
        is_from_aspect_reponse = is_from_aspect(row, question_col, answer_col)
        if is_from_aspect_reponse == 1:
            # update the count of aspect
            eval_dataframe["from_aspect"] += 1
            is_meaningful_reponse = is_meaningful(row, question_col, answer_col)
            if is_meaningful_reponse in [1, 2]:
                question_complexity_reponse = question_complexity(row, question_col, answer_col)
                is_answer_correct_reponse = is_answer_correct(row, question_col, answer_col)
                
                                
                # update the count of meaningfulness
                if is_meaningful_reponse == 1:
                    eval_dataframe["meaningful_1"] += 1
                else: eval_dataframe["meaningful_2"] += 1
                
                # update the count of complexity
                if question_complexity_reponse == 2:
                    eval_dataframe["complex"] += 1
                elif question_complexity_reponse == 1:
                    eval_dataframe["moderate"] += 1
                else: eval_dataframe["simple"] += 1
                
                #update correct answer
                if is_answer_correct_reponse == 2:
                    eval_dataframe["very_correct"] += 1
                elif is_answer_correct_reponse == 1:
                    eval_dataframe["partially_correct"] += 1
                else: eval_dataframe["incorrect"] += 1
                
                
            else: eval_dataframe["meaningful_0"] += 1
    
        else:
            eval_dataframe["not_from_aspect"] += 1
        
        return eval_dataframe
        
    for index, row in yes_no_questions.iterrows():
        question_col, answer_col = "Question", "Answer"
        eval_dataframe_yes_no = single_question_eval(row, question_col, answer_col, eval_dataframe_yes_no)
    
    for index, row in MCQ_S_questions.iterrows():
        question_col, answer_col = "Question & Options", "Correct Answer"
        eval_dataframe_MCQ_S = single_question_eval(row, question_col, answer_col, eval_dataframe_MCQ_S)
    
    for index, row in MCQ_M_questions.iterrows():
        question_col, answer_col = "Question & Options", "Correct Answer"
        eval_dataframe_MCQ_M = single_question_eval(row, question_col, answer_col, eval_dataframe_MCQ_M)
    
    
    return eval_dataframe_yes_no, eval_dataframe_MCQ_S, eval_dataframe_MCQ_M

In [ ]:
filmIndustry = "Hollywood"
nodeFile_root = "/mnt/Data/prabirmondal/prabir/python_program/movie_sense/SRI_KG/Movie_sense_KG/Movie_sense_KG/Dataset/Nodes_Edges/Hollywood"
questionFile_root = ""
evaluated_movie = pd.DataFrame(columns=['film_industry','category', 'movie_name', 'yor', 'aspect'])
node_eval_dataframe = pd.DataFrame(columns=["film_industry", "category", "aspect", "triplet_relevant", "triplet_irrelevant", "relation_relevant", "relation_irrelevant"])
question_eval_dataframe = pd.DataFrame(columns=["film_industry", "category", "complexity", "question_type", "aspect",  "from_aspect", "not_from_aspect", "meaningful_0", "meaningful_1", "meaningful_2", "very_correct", "partially_correct", "incorrect", "complex", "moderate", "simple"])


Categories = os.listdir(nodeFile_root)


aspect_list = ['Accolades',
'Cast',
'Guardians of the Galaxy Vol',
'Music',
'Plot',
'Production',
'Soundtrack',
'Themes',
'summary']


# define the dictionaries to keep the evaluations
#-------------------------------------------------

# for the node evaluation
#===============================
for category in Categories:
    for aspect in aspect_list:
        triplet_eval = {"film_industry": filmIndustry,
                        "category" : category,
                        "aspect" : aspect,
                        "triplet_relevant" : 0, 
                        "triplet_irrelevant" : 0,
                        "relation_relevant" : 0,
                        "relation_irrelevant" : 0}
        
# for the questions evaluations
#================================

        # for hop1------------------------------
        eval_dataframe_yes_no_h1 = {"film_industry": filmIndustry, "category" : category, "complexity" : "simple", "question_type" : "yes_no", "aspect" : aspect,  "from_aspect" : 0, "not_from_aspect" : 0, "meaningful_0" : 0, "meaningful_1" : 0, "meaningful_2" : 0, "very_correct" : 0, "partially_correct" : 0, "incorrect" : 0, "complex" : 0, "moderate" : 0, "simple" : 0}
    
        eval_dataframe_MCQ_S_h1 = {"film_industry": filmIndustry, "category" : category, "complexity" : "simple", "question_type" : "mcq_s", "aspect" : aspect,  "from_aspect" : 0, "not_from_aspect" : 0, "meaningful_0" : 0, "meaningful_1" : 0, "meaningful_2" : 0, "very_correct" : 0, "partially_correct" : 0, "incorrect" : 0, "complex" : 0, "moderate" : 0, "simple" : 0}
    
        eval_dataframe_MCQ_M_h1 = {"film_industry": filmIndustry, "category" : category, "complexity" : "simple", "question_type" : "mcq_m", "aspect" : aspect,  "from_aspect" : 0, "not_from_aspect" : 0, "meaningful_0" : 0, "meaningful_1" : 0, "meaningful_2" : 0, "very_correct" : 0, "partially_correct" : 0, "incorrect" : 0, "complex" : 0, "moderate" : 0, "simple" : 0}
        
        # for hop2------------------------------
        eval_dataframe_yes_no_h2 = {"film_industry": filmIndustry, "category" : category, "complexity" : "moderate", "question_type" : "yes_no", "aspect" : aspect,  "from_aspect" : 0, "not_from_aspect" : 0, "meaningful_0" : 0, "meaningful_1" : 0, "meaningful_2" : 0, "very_correct" : 0, "partially_correct" : 0, "incorrect" : 0, "complex" : 0, "moderate" : 0, "simple" : 0}
    
        eval_dataframe_MCQ_S_h2 = {"film_industry": filmIndustry, "category" : category, "complexity" : "moderate", "question_type" : "mcq_s", "aspect" : aspect,  "from_aspect" : 0, "not_from_aspect" : 0, "meaningful_0" : 0, "meaningful_1" : 0, "meaningful_2" : 0, "very_correct" : 0, "partially_correct" : 0, "incorrect" : 0, "complex" : 0, "moderate" : 0, "simple" : 0}
    
        eval_dataframe_MCQ_M_h2 = {"film_industry": filmIndustry, "category" : category, "complexity" : "moderate", "question_type" : "mcq_m", "aspect" : aspect,  "from_aspect" : 0, "not_from_aspect" : 0, "meaningful_0" : 0, "meaningful_1" : 0, "meaningful_2" : 0, "very_correct" : 0, "partially_correct" : 0, "incorrect" : 0, "complex" : 0, "moderate" : 0, "simple" : 0}
        
        # for hop3------------------------------
        eval_dataframe_yes_no_h3 = {"film_industry": filmIndustry, "category" : category, "complexity" : "complex", "question_type" : "yes_no", "aspect" : aspect,  "from_aspect" : 0, "not_from_aspect" : 0, "meaningful_0" : 0, "meaningful_1" : 0, "meaningful_2" : 0, "very_correct" : 0, "partially_correct" : 0, "incorrect" : 0, "complex" : 0, "moderate" : 0, "simple" : 0}
    
        eval_dataframe_MCQ_S_h3 = {"film_industry": filmIndustry, "category" : category, "complexity" : "complex", "question_type" : "mcq_s", "aspect" : aspect,  "from_aspect" : 0, "not_from_aspect" : 0, "meaningful_0" : 0, "meaningful_1" : 0, "meaningful_2" : 0, "very_correct" : 0, "partially_correct" : 0, "incorrect" : 0, "complex" : 0, "moderate" : 0, "simple" : 0}
    
        eval_dataframe_MCQ_M_h3 = {"film_industry": filmIndustry, "category" : category, "complexity" : "complex", "question_type" : "mcq_m", "aspect" : aspect,  "from_aspect" : 0, "not_from_aspect" : 0, "meaningful_0" : 0, "meaningful_1" : 0, "meaningful_2" : 0, "very_correct" : 0, "partially_correct" : 0, "incorrect" : 0, "complex" : 0, "moderate" : 0, "simple" : 0}

        
        
        # 8. add aspect in both dataframe of node and question
        
        node_file_names = os.listdir(os.path.join(nodeFile_root, category))
        node_file_names = [node_file_name for node_file_name in node_file_names if node_file_name.split(".")[0].split("_")[-1] == aspect]
        for node_file_name in node_file_names:
            # 9. check if the file already covered then continue otherwise procede
            
            # get the context from wikipedia
            movie_name, yor = node_file_name.split("_")[0], node_file_name.split("_")[1]
            print(movie_name)
            wiki_link = movie_links[movie_links.movie_name == movie_name]["wiki_link"].iloc[0]

            scraped_dict = scrape(wiki_link)
            
            try:
                node_file = pd.read_csv(os.path.join(nodeFile_root, category, node_file_name))

                sample_30_count = int(0.3*len(node_file))
                node_file = node_file.sample(n=sample_30_count, random_state=seed_value)
                
            except:
                
                continue
                
            if len(node_file)>=3:
                
                h1q1, h1q2, h1q3 = fetch_questions(category, node_file_name, "Simple_hop1")
                h2q1, h2q2, h2q3 = fetch_questions(category, node_file_name, "Moderate_hop2")
                h3q1, h3q2, h3q3 = fetch_questions(category, node_file_name, "Complex_hop3")
                
                try:
                    #save the scraped aspect of the considered movie                
                    save_aspect(aspect, scraped_dict[aspect], movie_name, yor)
                    clear_terminal()
                    print("\n Please check the content from the movie.txt file and then response the questions here. \nPress Enter to start.")
                    print("==================================================================================================================")                    
                    input()
                    
                                        
                    # 1. Evaluate nodes (use terminal clear in the eval method)
                    triplet_eval = eval_nodes(node_file, triplet_eval) #returns relevant and irrelevant triplet counts and relation counts
                    
                    # 2. update the node evaluation dataframe of this aspect..........done
                    
                    # 3. Evaluate Questions
                    eval_dataframe_yes_no_h1, eval_dataframe_MCQ_S_h1, eval_dataframe_MCQ_M_h1 = eval_question(h1q1, h1q2, h1q3, eval_dataframe_yes_no_h1, eval_dataframe_MCQ_S_h1, eval_dataframe_MCQ_M_h1)
                    eval_dataframe_yes_no_h2, eval_dataframe_MCQ_S_h2, eval_dataframe_MCQ_M_h2 = eval_question(h2q1, h2q2, h2q3, eval_dataframe_yes_no_h2, eval_dataframe_MCQ_S_h2, eval_dataframe_MCQ_M_h2)
                    eval_dataframe_yes_no_h3, eval_dataframe_MCQ_S_h3, eval_dataframe_MCQ_M_h3 = eval_question(h3q1, h3q2, h3q3, eval_dataframe_yes_no_h3, eval_dataframe_MCQ_S_h3, eval_dataframe_MCQ_M_h3)
                    
                    # 4. merge all the question evaluation
                    
                    # 5. update the question evaluation dataframe of this aspect
                    
                    df1 = [triplet_eval]
                    df1 = pd.DataFrame.from_dict(df1)
                    node_eval_dataframe = pd.concat([df1, node_eval_dataframe], axis=0, ignore_index=False)
                    
                    
                    df2 = [eval_dataframe_yes_no_h1, eval_dataframe_MCQ_S_h1, eval_dataframe_MCQ_M_h1, eval_dataframe_yes_no_h2, eval_dataframe_MCQ_S_h2, eval_dataframe_MCQ_M_h2, eval_dataframe_yes_no_h3, eval_dataframe_MCQ_S_h3, eval_dataframe_MCQ_M_h3 ]
                    df2 = pd.DataFrame.from_dict(df2)
                    question_eval_dataframe = pd.concat([df2, question_eval_dataframe], axis=0, ignore_index=False)
                    
                    
                    # 7. check the movie count for taking decision for further process
                    
                    
                except:
                    continue
                
            # 6. save the movie file name so that recompute is omit
                    
            evaluated_movie.loc[len(evaluated_movie)] = [filmIndustry, category, movie_name, yor, aspect]
                    
                    
            
        


Gods of Egypt

 Please check the content from the movie.txt file and then response the questions here
-----------------------------------------------------------------------------------------
Fantasy Island
Baywatch
Hot Pursuit
The Oogieloves in the Big Balloon Adventure

 Please check the content from the movie.txt file and then response the questions here
-----------------------------------------------------------------------------------------
In the Name of the King

 Please check the content from the movie.txt file and then response the questions here
-----------------------------------------------------------------------------------------
All About Steve

 Please check the content from the movie.txt file and then response the questions here
-----------------------------------------------------------------------------------------
Fifty Shades Freed

 Please check the content from the movie.txt file and then response the questions here
-----------------------------------------------

In [32]:
import pandas as pd 
	
# Initialise data to lists. 
data = [{'Geeks': 'dataframe', 'For': 'using', 'geeks': 'list'}] 

df = pd.DataFrame.from_dict(data) 
print(df)


       Geeks    For geeks
0  dataframe  using  list


In [29]:
import pandas as pd
# Create two sample DataFrames
df1 = pd.DataFrame({'Name': ['Alice', 'Bob'], 'Age': [25, 30]})
print(df1)
df2 = pd.DataFrame({'City': ['New York', 'Los Angeles'], 'Salary': [70000, 80000]})
print(df2)

# Combine using concat() along columns
combined_df = pd.concat([df1, df2], axis=0, ignore_index=False)
display(combined_df)

    Name  Age
0  Alice   25
1    Bob   30
          City  Salary
0     New York   70000
1  Los Angeles   80000


,Name,Age,City,Salary
0,Alice,25.0,NaN,NaN
1,Bob,30.0,NaN,NaN
0,NaN,NaN,New York,70000.0
1,NaN,NaN,Los Angeles,80000.0


In [ ]:
data = [{'Geeks': 'dataframe', 'For': 'using', 'geeks': 'list'}, 
		{'Geeks':10, 'For': 20, 'geeks': 30}] 
df = pd.DataFrame.from_dict(data)
# Combine using concat() along columns
combined_df = pd.concat([df1, df2], axis=0, ignore_index=False)

In [ ]:
# Send a request to the Wikipedia page
response = requests.get("https://en.wikipedia.org/wiki/Fantasy_Island_(film)")

# Parse the page content
soup = BeautifulSoup(response.content, 'html.parser')


scraped_dict = {}
    
#-------------------Summary scraping--------------------------
# Find the first paragraph after the title, which is usually the summary
summary_paragraphs = []
# Wikipedia's summary paragraphs are inside <p> tags but before any <h2> tag
for paragraph in soup.find_all('p'):
    
    # Ensure the paragraph has text and is not empty
    if paragraph.get_text().strip():
        summary_paragraphs.append(paragraph.get_text().strip())
    
    # Stop once we hit the first section heading (e.g. 'Plot' or 'Contents')
    if paragraph.find_next_sibling(['h2', 'h3']):
        break

scraped_dict["summary"] =  ' '.join(summary_paragraphs)
scraped_dict

# # -------------------Other Aspect Scraping---------------------
# Headings = soup.find_all('div', class_ = 'mw-heading mw-heading2')
# for heading in Headings:
#     try:
#         aspect, _ = (heading.text).split('[')
#     except:
#         aspect = heading.text
        
#     if aspect.lower() in Aspects:
#         next_siblings = heading.find_next_siblings()
#         text = ''
#         for next_sibling in next_siblings:
#             next_sibling_name = next_sibling.name
#             # sub_sibling = ''
#             # print(f"............next_sibling_name = {next_sibling_name}")
#             if (next_sibling_name =='style'):
#                 continue
            
#             elif (next_sibling_name == 'div'):
#                 clss = next_sibling.get('class')
                
#                 if ('mw-heading2' in clss):  # break because heading ended
#                     break
#             text += " "+ next_sibling.text
#         scraped_dict[aspect] = text 
# return scraped_dict

In [ ]:
import pandas as pd
import random

# Sample data for nouns
nouns = ["tree", "car", "river", "mountain", "city", "book", "ocean", "computer", "bird", "house"]

# Function to generate a sentence for the edge
def generate_edge(node1, node2):
    # torch.manual_seed(42)
    include_nodes = random.choice([True, False])  # Randomly decide whether to include nodes
    if include_nodes:
        # Include node1 and node2 in the edge sentence
        return f"{node1} is connected to {node2} in some way."
    else:
        # Generate a random sentence without mentioning node1 and node2
        sentences = [
            "There is a strong relationship here.",
            "The connection is abstract but meaningful.",
            "This is a symbolic link.",
            "A significant association exists.",
            "A conceptual bond is present."
        ]
        return random.choice(sentences)

# Generate 10 rows for the DataFrame
data = []
for _ in range(10):
    node1, node2 = random.sample(nouns, 2)  # Randomly select two different nouns
    edge = generate_edge(node1, node2)     # Generate the edge sentence
    data.append({"node_1": node1, "node_2": node2, "edge": edge})

# Create the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


In [5]:
import pandas as pd
import numpy as np

def rndm(df):
    # Randomly choose rows
    n = 3  # Number of rows to select
    df = df.sample(n=n, random_state=seed_value)
    return df
    

# Example DataFrame
data = {
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50],
    'C': [100, 200, 300, 400, 500]
}
df = pd.DataFrame(data)
print(df)

# Set the seed value for reproducibility
seed_value = 42
np.random.seed(seed_value)


df = rndm(df)

print("Randomly selected rows:")
print(df)


   A   B    C
0  1  10  100
1  2  20  200
2  3  30  300
3  4  40  400
4  5  50  500
Randomly selected rows:
   A   B    C
1  2  20  200
4  5  50  500
2  3  30  300


In [ ]:
pd_file = pd.read_csv("/mnt/Data/prabirmondal/prabir/python_program/movie_sense/SRI_KG/Movie_sense_KG/Movie_sense_KG/Dataset/Nodes_Edges/Hollywood/Least_Popular/All About Steve_2009_Plot.csv")

# print(pd_file.head())

print(len(pd_file))
pd_file.columns

pd_file = preprocess_node_df(pd_file)
pd_file.head()

# len(pd_file)
# pd_file.columns

In [ ]:
pd_file = pd_file.sample(n=4)
pd_file.head()

In [ ]:

for index, row in pd_file.iterrows():
    node1 = row["node_1"]
    node2 = row["node_2"]
    edge = row["edge"]
    print(row["firstname"])